# Carga de datos

In [0]:
#Conecta el notebook con el Datalake
storage_account_name="YourStorageaccountname"
container_name="YourLandingContainer"
datalake_key="Key"

spark.conf.set("fs.azure.account.key.{}.dfs.core.windows.net".format(storage_account_name),datalake_key)

#Declara la ubicación de los datos a cargar
data_location = "abfss://{}@{}.dfs.core.windows.net/".format(container_name,storage_account_name)

#Muestra los elementos en el container
dbutils.fs.ls(data_location)

Out[1]: [FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/Categoria.csv', name='Categoria.csv', size=392, modificationTime=1669729477000),
 FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/FactMine.csv', name='FactMine.csv', size=392, modificationTime=1669729476000),
 FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/Mine.csv', name='Mine.csv', size=392, modificationTime=1669729474000),
 FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/Producto.csv', name='Producto.csv', size=392, modificationTime=1669729475000),
 FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/Stock.csv', name='Stock.csv', size=392, modificationTime=1669729474000),
 FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/SubCategoria.csv', name='SubCategoria.csv', size=392, modificationTime=1669729475000),
 FileInfo(path='abfss://input@almacendatalake.dfs.core.windows.net/Sucursales.csv', name='Sucursales.csv', size=392, mod

In [0]:
#Guarda la ubicación dentro del container de cada archivo csv
file_location1= data_location + "Categoria.csv"
file_location2= data_location + "FactMine.csv"
file_location3= data_location + "Mine.csv"
file_location4= data_location + "Producto.csv"
file_location5= data_location + "SubCategoria.csv"
file_location6= data_location + "Sucursales.csv"
file_location7= data_location + "VentasInternet.csv"
file_location8= data_location + "Stock.csv"

In [0]:
#Carga los archivos csv a dataframes
categoria= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location1)
factmine= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location2)
mine= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location3)
prod= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location4)
subcat= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location5)
sucursales= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location6)
ventasinternet= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location7)
stock= spark.read.format("csv").option("inferSchema", "true").option("header","true").option("delimiter",",").load(file_location8)

In [0]:
display(stock)

TABLE_CATALOG TABLE_SCHEMA TABLE_NAME TABLE_TYPE dbRetail dbo Sucursales BASE TABLE dbRetail dbo FactMine BASE TABLE dbRetail dbo SubCategoria BASE TABLE dbRetail dbo Mine BASE TABLE dbRetail dbo Categoria BASE TABLE dbRetail dbo Producto BASE TABLE dbRetail dbo VentasInternet BASE TABLE dbRetail dbo Stock BASE TABLE

# Transformaciones

In [0]:
#Declaramos el nombre del container al que se guardan los archivos de salida, que debe estar creado previamente o usar el mismo del input
output_container_name = "outputcontainer"
output_location = "abfss://{}@{}.dfs.core.windows.net/".format(output_container_name,storage_account_name)

In [0]:
#Se define una función para guardar los archivos
def escribirCSV(dataFrame,nombre,removeFolder=False):
    filename = nombre[:1].lower()+nombre[1::] #Cambia a miníscula la primera letra para el nombre del archivo
    foldername= filename.capitalize()  #Cambia a mayúscula la primera letra para el nombre de la carpeta
    
    dataFrame.coalesce(1)\
    .write.format("com.databricks.spark.csv")\
    .mode('overwrite')\
    .option("header", "true")\
    .save(output_location+foldername)
    #Colisiona  la tabla en una sola partición
    #Guarda el archivo como csv
    #Sobreescribe si ya existe
    #Conserva el nombre de las columnas en la primera fila
    #Guarda en la carpeta con el nombre indicado en el output container

    #La función write crea una carpeta dónde el archivo csv se encuentra como 'part-00000-tid-xxxxxx.csv', hay que encontrarlo y moverlo
    for file in dbutils.fs.ls(output_location+foldername): #Busca en los archivos de la carpeta creada
        if file.name.endswith(".csv"): #Se asegura de que sea el archivo .csv
            print("Se movió {} a la raiz como {}".format(file.name,filename+".csv"))
            dbutils.fs.mv(file.path,output_location+filename+".csv") #Y si es lo mueve y  cambia el nombre al archivo.csv
    
    if(removeFolder):#Si se le dice que remueva la carpeta, trata de eliminar la carpeta que creó
        dbutils.fs.rm(output_location+foldername,True)
        print("Se eliminó la carpeta {}".format(foldername))

## Transformación 1
En la tabla “Categoria” renombrar el campo Categoria a Nombre_Categoria.

In [0]:
#Se guarda en categoria la tabla con el nuevo nombre de la columna
categoria = categoria.withColumnRenamed("Categoria","Nombre_Categoria")
display(categoria)

Cod_Categoria Nombre_Categoria 1 Bicicleta 2 Componente 3 Prenda 4 Accesorio

In [0]:
#Se guarda en el uotput container la nueva tabla
escribirCSV(categoria,"Categoria",True)

Se movió part-00000-tid-3117304753624374715-eada6129-4718-42f6-bb85-e6028c14f3ef-104-1-c000.csv a la raiz como categoria.csv
Se eliminó la carpeta Categoria

## Transformación 2
En la tabla “FactMine” obtener el total de la columna: TotalOreMined.

In [0]:
display(factmine.agg({'TotalOreMined':'sum'}))

sum(TotalOreMined) 166599.50999999998

## Transformación 3
En la tabla “Mine” seleccionar únicamente las columnas: Country, FirstName,LastName y Age.

In [0]:
#Se selecciona únicamente las columnas Country, FirstName,LastName y Age
mineSelected = mine.select("Country", "FirstName", "LastName", "Age")
display(mineSelected)

Country FirstName LastName Age Australia Curtis Lu 41 Australia Curtis Lu 41 Australia Curtis Lu 41 Australia Jill Jimenez 59 Australia Jill Jimenez 59 Australia Jill Jimenez 59 Australia Bethany Yuan 52 Australia Bethany Yuan 52 Australia Bethany Yuan 52 Australia Jon Zhou 51 Australia Jon Zhou 51 Australia Jon Zhou 51 United States Noah Powell 29 Australia Leah Ye 47 Australia Leah Ye 47 Australia Leah Ye 47 Australia Andrés Anand 42 Australia Andrés Anand 42 Australia Andrés Anand 42 Australia Meredith Gutierrez 43 Australia Meredith Gutierrez 43 Australia Meredith Gutierrez 43 United States Jasmine Coleman 25 Australia Ernest Wu 66 Australia Ernest Wu 66 Australia Ernest Wu 66 United States Maurice Tang 31 United States Chase Cox 31 Germany Ricky Vazquez 48 Germany Ricky Vazquez 48 Germany Ricky Vazquez 48 Germany Ricky Vazquez 48 United Kingdom Claudia Zhou 49 United Kingdom Claudia Zhou 49 United Kingdom Claudia Zhou 49 United Kingdom Claudia Zhou 49 United States Eduardo Martin 41 United States Eduardo Martin 41 United States Haley Richardson 69 United States Haley Richardson 69 Canada Julian Griffin 65 Canada Julian Griffin 65 France Robin Ramos 48 France Robin Ramos 48 Australia Jennifer Collins 25 Australia Jennifer Collins 25 Australia Jennifer Collins 25 France Gary Vazquez 34 France Gary Vazquez 34 United Kingdom Meredith Raman 40 United Kingdom Meredith Raman 40 United Kingdom Meredith Raman 40 United Kingdom George McDonald 35 United Kingdom George McDonald 35 United Kingdom George McDonald 35 United Kingdom Megan Stewart 51 United Kingdom Megan Stewart 51 United Kingdom Megan Stewart 51 United Kingdom Megan Stewart 51 Germany Deanna Perez 51 Germany Deanna Perez 51 Germany Deanna Perez 51 Germany Deanna Perez 51 Australia Kari Kim 30 Australia Kari Kim 30 Australia Kari Kim 30 Australia Jennifer Roberts 30 Australia Jennifer Roberts 30 Australia Jennifer Roberts 30 Australia Laura Lin 33 Australia Laura Lin 33 Australia Laura Lin 33 Australia Alejandro Huang 33 Australia Alejandro Huang 33 Australia Alejandro Huang 33 France Darryl Wu 32 France Darryl Wu 32 France Darryl Wu 32 Germany Erica Hu 32 Germany Erica Hu 32 Germany Erica Hu 32 United Kingdom Jimmy Gutierrez 34 United Kingdom Jimmy Gutierrez 34 United Kingdom Jimmy Gutierrez 34 United Kingdom Gary Ortega 34 United Kingdom Gary Ortega 34 United Kingdom Gary Ortega 34 Canada Alyssa Howard 45 United States Joan Washington 42 United States Joan Washington 42 France Cindy Sai 31 France Cindy Sai 31 France Cindy Sai 31 Germany Jorge Liang 38 Germany Jorge Liang 38 Germany Meredith Ruiz 27 Germany Meredith Ruiz 27 France Ernest Lin 25 France Ernest Lin 25 Germany Meredith Prasad 29 Germany Meredith Prasad 29 United Kingdom Dwayne Navarro 29 United Kingdom Dwayne Navarro 29 United Kingdom Dwayne Navarro 29 United States Mariah Wood 33 United States Mariah Wood 33 United States Haley Powell 33 United States Haley Powell 33 Canada Randy Wu 35 United States Alyssa Morgan 35 United States Alyssa Morgan 35 United States Mackenzie Campbell 35 United States Mackenzie Campbell 35 United States Allison Gonzalez 46 United States Allison Gonzalez 46 Australia Randall Gomez 26 Australia Randall Gomez 26 Australia Randall Gomez 26 Australia Jessie Ramos 37 Australia Jessie Ramos 37 Australia Jessie Ramos 37 United States Sarah Jackson 33 United States Sarah Jackson 33 United States Isabel Jenkins 25 United States Isabel Jenkins 25 United States Brendan Chande 36 United States Brendan Chande 36 United States Mary Patterson 37 United States Mary Patterson 37 United States Hannah Patterson 29 United States Hannah Patterson 29 United States Taylor Garcia 36 United States Taylor Garcia 36 Australia Frank Carlson 35 Australia Frank Carlson 35 Australia Frank Carlson 35 Australia Angela Henderson 35 Australia Angela Henderson 35 Australia Angela Henderson 35 Australia Philip Carlson 37 Australia Philip Carlson 37 Australia Philip Carlson 37 Australia Jaclyn Nara 58 Australia Jacly

In [0]:
#Se guarda la nueva tabla como mineSelected.csv
escribirCSV(mineSelected,"MineSelected",True)

Se movió part-00000-tid-3033866231641724643-16f6058c-a118-4ecd-b0fb-e2143f18bbd5-108-1-c000.csv a la raiz como mineSelected.csv

## Transformación 4
En la tabla “Mine” obtener el total de la columna: TotalWasted agrupado por el atributo Country.

In [0]:
#Se calcula el total gastato por país
TotalWasted_byCountry = mine.groupBy("Country").sum("TotalWasted").withColumnRenamed("sum(TotalWasted)","TotalWastedByCountry")
display(TotalWasted_byCountry)

Country TotalWastedByCountry Germany 224681.15220000016 France 204286.06200000053 United States 719988.7782999972 Canada 145704.19389999963 Australia 708164.0202000095 United Kingdom 262627.4179000008

In [0]:
#Se guarda la nueva tabla como totalWastedByCountry.csv
escribirCSV(TotalWasted_byCountry,"TotalWastedByCountry",True)

Se movió part-00000-tid-5254959478874537621-acc1818a-536d-4830-8fab-d4ae09c05c18-112-1-c000.csv a la raiz como totalWastedByCountry.csv

## Transformación 5
En la tabla “Producto” realizar un conteo de la cantidad de productos disponibles utilizando el campo: Cod_Producto.

In [0]:
count_cod_product = prod.select('Cod_Producto').count()
print(count_cod_product)

319

## Transformación 6
 En la tabla “Producto” mostrar de forma descendiente la información ordenada por el campo: Cod_Producto.

In [0]:
from pyspark.sql.functions import desc

#Se guarda en prod la tabla con el nuevo orden
prod = prod.sort(desc("Cod_Producto"))
display(prod)

Cod_Producto Producto Cod_SubCategoria Color 606 Carretera: 750, negra, 52 2 Black 605 Carretera: 750, negra, 48 2 Black 604 Carretera: 750, negra, 44 2 Black 603 Eje de pedalier GA 5 NA 602 Eje de pedalier GM 5 NA 601 Eje de pedalier GB 5 NA 600 Montaña: 500, negra, 52 1 Black 599 Montaña: 500, negra, 48 1 Black 598 Montaña: 500, negra, 44 1 Black 597 Montaña: 500, negra, 42 1 Black 596 Montaña: 500, negra, 40 1 Black 595 Montaña: 500, plateada, 52 1 Silver 594 Montaña: 500, plateada, 48 1 Silver 593 Montaña: 500, plateada, 44 1 Silver 592 Montaña: 500, plateada, 42 1 Silver 591 Montaña: 500, plateada, 40 1 Silver 590 Montaña: 400 para mujer, plateada, 46 1 Silver 589 Montaña: 400 para mujer, plateada, 42 1 Silver 588 Montaña: 400 para mujer, plateada, 40 1 Silver 587 Montaña: 400 para mujer, plateada, 38 1 Silver 586 Paseo: 3000, azul, 50 3 Blue 585 Paseo: 3000, azul, 44 3 Blue 584 Carretera: 750, negra, 58 2 Black 583 Carretera: 350 para mujer, amarilla, 48 2 Yellow 582 Carretera: 350 para mujer, amarilla, 44 2 Yellow 581 Carretera: 350 para mujer, amarilla, 42 2 Yellow 580 Carretera: 350 para mujer, amarilla, 40 2 Yellow 579 Paseo: 2000, azul, 54 3 Blue 578 Paseo: 2000, azul, 50 3 Blue 577 Paseo: 2000, azul, 46 3 Blue 576 Paseo: 1000, azul, 60 3 Blue 575 Paseo: 1000, azul, 54 3 Blue 574 Paseo: 1000, azul, 50 3 Blue 573 Paseo: 1000, azul, 46 3 Blue 572 Paseo: 3000, amarilla, 62 3 Yellow 571 Paseo: 3000, amarilla, 58 3 Yellow 570 Paseo: 3000, amarilla, 54 3 Yellow 569 Paseo: 3000, amarilla, 50 3 Yellow 568 Paseo: 3000, amarilla, 44 3 Yellow 567 Paseo: 3000, azul, 62 3 Blue 566 Paseo: 3000, azul, 58 3 Blue 565 Paseo: 3000, azul, 54 3 Blue 564 Paseo: 1000, amarilla, 60 3 Yellow 563 Paseo: 1000, amarilla, 54 3 Yellow 562 Paseo: 1000, amarilla, 50 3 Yellow 561 Paseo: 1000, amarilla, 46 3 Yellow 560 Paseo: 2000, azul, 60 3 Blue 559 Cadena 7 Silver 558 Bielas GA 8 Black 557 Bielas GM 8 Black 556 Bielas GB 8 Black 555 Frenos delanteros 6 Silver 554 Manillar de paseo GA 4 NA 553 Manillar de paseo GB 4 NA 552 Desviador delantero 9 Silver 548 Pedal de paseo 13 Silver/Black 547 Pedal de carretera GA 13 Silver/Black 546 Pedal de carretera GM 13 Silver/Black 545 Pedal de carretera GB 13 Silver/Black 544 Pedal de montaña GA 13 Silver/Black 543 Pedal de montaña GM 13 Silver/Black 542 Pedal de montaña GB 13 Silver/Black 541 Cubierta de paseo 37 NA 540 Cubierta de carretera GA 37 NA 539 Cubierta de carretera GM 37 NA 538 Cubierta de carretera GB 37 NA 537 Cubierta de montaña GA 37 NA 536 Cubierta de montaña GM 37 NA 535 Cubierta de montaña GB 37 NA 530 Cubierta para cámara de paseo 37 NA 529 Cubierta para cámara de carretera 37 NA 528 Cubierta para cámara de montaña 37 NA 523 Sillín/asiento de paseo GA 15 NA 522 Sillín/asiento de paseo GM 15 NA 521 Sillín/asiento de paseo GB 15 NA 520 Sillín/asiento de carretera GA 15 NA 519 Sillín/asiento de carretera GM 15 NA 518 Sillín/asiento de carretera GB 15 NA 514 Frenos traseros 6 Silver 510 Cuadro de paseo GB: azul, 44 16 Blue 509 Cuadro de paseo GB: amarillo, 58 16 Yellow 508 Cuadro de paseo GB: amarillo, 54 16 Yellow 507 Cuadro de paseo GB: amarillo, 50 16 Yellow 506 Cuadro de paseo GB: amarillo, 44 16 Yellow 505 Cuadro de paseo GB: azul, 62 16 Blue 504 Cuadro de paseo GB: azul, 58 16 Blue 503 Cuadro de paseo GB: azul, 54 16 Blue 502 Cuadro de paseo GB: azul, 50 16 Blue 501 Desviador trasero 9 Silver 500 Cuadro de paseo GA: azul, 60 16 Blue 499 Cuadro de paseo GA: azul, 54 16 Blue 498 Cuadro de paseo GA: azul, 50 16 Blue 497 Cuadro de paseo GA: azul, 46 16 Blue 496 Cuadro de paseo GA: amarillo, 54 16 Yellow 495 Cuadro de paseo GA: amarillo, 50 16 Yellow 494 Cuadro de paseo GA: amarillo, 46 16 Yellow 493 Cuadro de paseo GB: amarillo, 62 16 Yellow 492 Cuadro de paseo GA: amarillo, 60 16 Yellow 491 Jersey clásico de manga corta, SG 21 Yellow 490 Jersey clásico de manga corta, G 21 Yellow 489 Jersey clásico de manga corta, M 21 Yellow 488 Jersey clásico de manga corta, P 21 Yellow 486 Soporte multius

In [0]:
#Se guarda la nueva tabla como producto.csv
escribirCSV(prod,"producto",True)

Se movió part-00000-tid-6839651390785425300-a054085c-7779-4f27-a04b-7517faa31c06-118-1-c000.csv a la raiz como producto.csv
Se eliminó la carpeta Producto

## Transformación 7
 En la tabla “Subcategoria” realizar un filtro por el Cod_Categoria = 3.

In [0]:
#se filtran las sibcategorías por la categoría 3
subcat3 = subcat.filter(subcat.Cod_Categoria == "3")
display(subcat3)

Cod_SubCategoria SubCategoria Cod_Categoria 18 Culote corto 3 19 Gorra 3 20 Guantes 3 21 Jersey 3 22 Pantalones cortos 3 23 Calcetines 3 24 Mallas 3 25 Camiseta 3

In [0]:
#Se guarda la nueva tabla como subCategoria3.csv
escribirCSV(subcat3,"subCategoria3",True)

Se movió part-00000-tid-6219436160814538263-3912b672-7b42-4297-b3a2-2cfd07e9e34c-120-1-c000.csv a la raiz como subCategoria3.csv
Se eliminó la carpeta Subcategoria3

## Transformación 8
En la tabla “VentasInternet” generar una columna llamada Ingresos Netos, que se obtenga de multiplicar los atributos: (Cantidad * PrecioUnitario) – CostoUnitario.

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as T

ingresos_Netos = ventasinternet.withColumn("Ingresos Netos", (F.col("Cantidad")*F.col("PrecioUnitario")) - F.col("CostoUnitario"))
display(ingresos_Netos)

Cod_Producto Cod_Cliente Cod_Territorio NumeroOrden Cantidad PrecioUnitario CostoUnitario Impuesto Flete FechaOrden FechaEnvio FechaVencimiento Cod_Promocion Ingresos Netos 528 12097 6 SO65573 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 535 26198 4 SO65574 1 24.99 9.3463 1.9992 0.6248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 15.643699999999999 536 11276 6 SO65575 1 29.99 11.2163 2.3992 0.7498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 18.773699999999998 528 11276 6 SO65575 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 536 15413 6 SO65576 1 29.99 11.2163 2.3992 0.7498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 18.773699999999998 528 15413 6 SO65576 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 214 15413 6 SO65576 1 34.99 13.0863 2.7992 0.8748 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 21.9037 234 15413 6 SO65576 1 49.99 38.4923 3.9992 1.2498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 11.497700000000002 478 13303 6 SO65577 1 9.99 3.7363 0.7992 0.2498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 6.2537 478 21292 1 SO65578 1 9.99 3.7363 0.7992 0.2498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 6.2537 478 22340 4 SO65579 1 9.99 3.7363 0.7992 0.2498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 6.2537 472 19636 1 SO65580 1 63.5 23.749 5.08 1.5875 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 39.751000000000005 473 24322 6 SO65582 1 63.5 23.749 5.08 1.5875 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 39.751000000000005 528 16369 1 SO65584 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 485 16369 1 SO65584 1 21.98 8.2205 1.7584 0.5495 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 13.759500000000001 528 16501 1 SO65585 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 528 22730 6 SO65586 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 485 14841 8 SO65587 1 21.98 8.2205 1.7584 0.5495 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 13.759500000000001 472 14841 8 SO65587 1 63.5 23.749 5.08 1.5875 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 39.751000000000005 528 14691 7 SO65588 1 4.99 1.8663 0.3992 0.1248 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 3.1237000000000004 222 14691 7 SO65588 1 34.99 13.0863 2.7992 0.8748 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 21.9037 234 14691 7 SO65588 1 49.99 38.4923 3.9992 1.2498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 11.497700000000002 540 14182 8 SO65589 1 32.6 12.1924 2.608 0.815 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 20.407600000000002 536 17822 7 SO65590 1 29.99 11.2163 2.3992 0.7498 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000+0000 1 18.773699999999998 538 12304 10 SO65591 1 21.49 8.0373 1.7192 0.5373 2013-09-02T00:00:00.000+0000 2013-09-09T00:00:00.000+0000 2013-09-14T00:00:00.000

In [0]:
#Se guarda la nueva tabla como ventasInternet.csv
escribirCSV(ingresos_Netos,"VentasInternet",True)

Se movió part-00000-tid-3356083071265953182-a271a29c-348e-4c19-b19c-cb78c3cbb486-122-1-c000.csv a la raiz como ventasInternet.csv
Se eliminó la carpeta Ventasinternet

## Transformación 9
En la tabla “VentasInternet” mostrar el promedio y suma total de la columna: Ingresos Netos por Cod_Producto.

In [0]:
from pyspark.sql.functions import sum, avg
from pyspark.sql import SparkSession
  
average_sum_byCodProducto = Ingresos_Netos.groupBy("Cod_Producto").agg(avg("Ingresos Netos"), sum("Ingresos Netos"))\
.withColumnRenamed("avg(Ingresos Netos)","PromedioIngresosNetos")\
.withColumnRenamed("sum(Ingresos Netos)","TotalIngresosNetos")
display(average_sum_byCodProducto)

Cod_Producto PromedioIngresosNetos TotalIngresosNetos 471 39.75100000000007 6678.1680000000115 540 20.407600000000244 17509.72080000021 580 618.4800000000007 152146.08000000016 588 349.7115999999998 44763.084799999975 481 5.6277000000000035 1677.054600000001 472 39.75100000000008 7910.449000000017 322 285.9519 4861.1823 362 943.2882000000019 189600.92820000037 321 296.2834000000002 16295.587000000012 375 860.8787000000009 121383.89670000013 593 256.7721000000001 10014.111900000004 597 245.41029999999984 12025.104699999993 530 3.1237000000000514 4648.065600000076 368 924.5635999999975 133137.15839999964 385 394.7883000000001 27240.392700000008 596 245.41029999999986 11779.694399999993 587 349.71159999999963 51757.31679999995 332 285.9519 5147.1342 577 459.69919999999945 44590.822399999946 384 407.4101999999992 81074.62979999984 606 196.34040000000036 75787.39440000014 329 296.2834000000002 18369.570800000012 336 285.9519 6290.9418000000005 350 1476.895600000002 88613.73600000012 388 407.4101999999991 85556.14199999982 578 459.6991999999994 48728.115199999935 333 296.2834000000002 17184.43720000001 222 21.903699999999912 46545.362499999814 330 285.9519 5719.0380000000005 372 888.4021000000006 119045.88140000007 604 196.3404000000003 70682.54400000011 539 15.64369999999989 14486.0661999999 346 1487.8355999999997 72903.94439999998 360 943.2882000000012 176394.89340000023 599 245.4102999999998 13742.97679999999 355 1054.3704999999975 413313.235999999 353 1054.3704999999968 444944.3509999987 386 407.41019999999907 96556.21739999978 325 296.2834000000002 17480.72060000001 473 39.75100000000008 7751.445000000016 581 618.4800000000009 145342.80000000022 363 1043.0087000000035 437020.6453000015 348 1476.8956000000017 72367.88440000008 572 280.9052000000001 14045.260000000006 328 285.9519 7434.749400000001 359 1043.0087000000037 433891.6192000015 479 5.6277000000000035 9634.622400000006 340 285.9519 4861.1823 377 860.878700000001 148071.13640000016 582 618.480000000001 133591.68000000023 570 280.9052000000001 13483.449600000005 339 296.2834000000002 12443.902800000009 491 12.417699999999977 5078.8392999999905 560 459.6991999999995 37235.63519999996 579 459.69919999999945 40453.52959999995 598 245.4102999999998 14233.797399999989 484 4.9767000000000055 4518.843600000005 347 1487.8355999999992 53562.08159999997 327 296.2834000000002 14814.170000000011 563 902.1321000000013 142536.8718000002 358 943.2882000000002 156585.84120000002 489 12.417699999999977 5054.003899999991 590 349.71159999999975 48260.20079999996 383 394.7883000000001 26450.816100000007 486 99.53399999999979 24783.96599999995 564 902.1321000000011 126298.49400000015 374 888.4021000000013 126153.09820000018 231 11.49769999999994 5081.983399999974 324 285.9519 4575.2304 586 280.9052000000001 13483.449600000005 538 13.452699999999885 14044.61879999988 387 394.7883000000001 30398.69910000001 381 394.7883000000001 28424.757600000008 583 618.4800000000009 143487.36000000022 594 256.7721000000001 12838.605000000005 485 13.759500000000092 29183.899500000196 310 1406.9758000000106 472743.86880000355 568 280.9052000000001 16573.406800000004 217 21.90369999999993 45669.214499999856 537 21.90999999999988 30586.359999999833 326 285.9519 6290.9418000000005 380 888.4021000000004 111938.66460000005 314 1406.97580000001 415057.86100000294 565 280.9052 15449.785999999998 343 296.2834000000002 16591.87040000001 585 280.90520000000004 14887.975600000002 331 296.2834000000002 12147.619400000009 234 11.49769999999994 5196.960399999973 312 1406.9758000000106 474150.84460000356 389 394.7883000000001 28424.757600000008 361 1043.0087000000035 445364.71490000153 589 349.7115999999998 45112.79639999998 569 280.9052000000001 16573.406800000004 320 285.9519 5433.0861 576 902.1321000000011 132613.41870000015 356 953.5636999999989 170687.9022999998 591 256.7721000000001 11554.744500000004 237 11.497699999999943 4748.550099999977 351 1476.8956000000019 84183.04920000011 482 5.6277000000000035 1519.479000000

In [0]:
#Se guarda la nueva tabla como ingresosPorProducto.csv
escribirCSV(average_sum_byCodProducto,"IngresosPorProducto",True)

Se movió part-00000-tid-5684276292585449993-982871cb-de97-4106-9403-7fb96b73d894-128-1-c000.csv a la raiz como ingresosPorProducto.csv
Se eliminó la carpeta Ingresosporproducto